
# Robustness Metrics demo


## Setup

In [7]:
from evo.core import metrics
from evo.core.units import Unit
from evo.tools import log
log.configure_logging(verbose=True, debug=True, silent=False)
import pprint
import numpy as np
from evo.tools import plot
import matplotlib.pyplot as plt
from evo.tools.settings import SETTINGS
import torch
import sys
sys.path.insert(1, '../script')


[DEBUG][2024-08-08 00:01:24,608][log.configure_logging():113]
System info:
Python 3.8.10
Linux-5.4.0-190-generic-x86_64-with-glibc2.29
zhh@zhh-MS-7E07



**Load two example trajectory files in TUM format;**

In [8]:
from evo.tools import file_interface

ref_file = "../test/data/freiburg1_xyz-groundtruth.txt"
est_file = "../test/data/freiburg1_xyz-rgbdslam_drift.txt"

traj_ref = file_interface.read_tum_trajectory_file(ref_file)
traj_est = file_interface.read_tum_trajectory_file(est_file)


print(f"Length of reference trajectory: {len(traj_ref.positions_xyz)}")
print(f"Length of estimated trajectory: {len(traj_est.positions_xyz)}")

[DEBUG][2024-08-08 00:01:24,652][file_interface.read_tum_trajectory_file():118]
Loaded 3000 stamps and poses from: ../test/data/freiburg1_xyz-groundtruth.txt
[DEBUG][2024-08-08 00:01:24,656][file_interface.read_tum_trajectory_file():118]
Loaded 788 stamps and poses from: ../test/data/freiburg1_xyz-rgbdslam_drift.txt
Length of reference trajectory: 3000
Length of estimated trajectory: 788


 **Obtain pairs via matching timestamps:**

In [9]:
from evo.core import sync

max_diff = 0.01
traj_ref, traj_est = sync.associate_trajectories(traj_ref, traj_est, max_diff)
print(f"Length of reference trajectory: {len(traj_ref.positions_xyz)}")
print(f"Length of estimated trajectory: {len(traj_est.positions_xyz)}")

[DEBUG][2024-08-08 00:01:24,664][sync.associate_trajectories():110]
Found 785 of max. 788 possible matching timestamps between...
	first trajectory
and:	second trajectory
..with max. time diff.: 0.01 (s) and time offset: 0.0 (s).
Length of reference trajectory: 785
Length of estimated trajectory: 785


**Plot the trajectories:**

In [10]:
# fig = plt.figure()
# traj_by_label = {
#     "estimate (not aligned)": traj_est,
#     "reference": traj_ref
# }
# plot.trajectories(fig, traj_by_label, plot.PlotMode.xyz)
# plt.show()

**F score calculation:**

In [11]:
from robustness import RobustnessMetric

# Convert the positions to the format expected by RobustnessMetric
trans_deriv1 = [torch.tensor(pos) for pos in traj_est.positions_xyz]
trans_deriv2 = [torch.tensor(pos) for pos in traj_ref.positions_xyz]

# For simplicity, assume rotational derivatives are zero (as they are not provided in the example)
rot_deriv1 = [torch.tensor([0.0, 0.0, 0.0]) for _ in traj_est.positions_xyz]
rot_deriv2 = [torch.tensor([0.0, 0.0, 0.0]) for _ in traj_ref.positions_xyz]

trans_threshold = 0.1
rot_threshold = 0.1


fscore_trans, fscore_rot = RobustnessMetric.calc_fscore(
    trans_deriv1, rot_deriv1, trans_deriv2, rot_deriv2, trans_threshold, rot_threshold
)

print(f"F-score for translation: {fscore_trans} for rotation: {fscore_rot}")




F-score for translation: 0.2878980891719745 for rotation: 1.0


**AUC calculation**

In [12]:
from robustness import RobustnessMetric
# Thresholds
threshold_start = 0.0
threshold_end = 1.0
threshold_interval = 0.1

## calculate AUC
result = RobustnessMetric.eval_robustness_batch(
    trans_deriv1, rot_deriv1, trans_deriv2, rot_deriv2,
    threshold_start, threshold_end, threshold_interval
)

print(result)



NameError: name 'asdasdasd' is not defined